In [ ]:
from transformers import pipeline, AutoTokenizer, BertForQuestionAnswering, BertForMaskedLM
from datasets import load_dataset

# Load the SQuAD dataset
squad_dataset = load_dataset("pubmed_qa", "pqa_unlabeled")
raw_datasets = squad_dataset

raw_datasets

In [ ]:
# Load the pre-trained model and tokenizer
model_name = "adsabs/astroBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["question"],  examples["long_answer"], padding="max_length", truncation=True, max_length=206)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("question", "text")
tokenized_datasets = tokenized_datasets.rename_column("long_answer", "labels")

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]

In [ ]:
full_train_dataset = full_train_dataset.map(
    lambda example: {"labels": example["context"]["labels"]},
)

In [ ]:
print(full_train_dataset.column_names)
print(full_train_dataset.features["input_ids"])

In [ ]:
from transformers import Trainer, TrainingArguments
# Define training arguments
training_args = TrainingArguments(
    output_dir="./qa_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
)



# training_args = TrainingArguments("test_trainer")

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels),

In [ ]:

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./qa_finetuned")
tokenizer.save_pretrained("./qa_finetuned")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()